In [1]:
from pathlib import Path

import typer
import pandas as pd
import verifiers as vf
from dotenv import load_dotenv
from openai import AsyncOpenAI


assert load_dotenv(), "Failed to load .env file"

In [2]:
import mlflow

# Tell MLflow about the server URI.
mlflow.set_tracking_uri("http://127.0.0.1:5000")
# Enable autologging with all features
mlflow.openai.autolog()
# Create a unique name for your experiment.
mlflow.set_experiment("rlvr-eval")

<Experiment: artifact_location='mlflow-artifacts:/2', creation_time=1756636926792, experiment_id='2', last_update_time=1756636926792, lifecycle_stage='active', name='rlvr-eval', tags={}>

In [3]:
openai_client = AsyncOpenAI()

models_response = await openai_client.models.list()
available_models = [item.id for item in models_response.data if item.object == "model"]
print("Available models:", available_models)

model = available_models[0]
print("Using model:", model)


Available models: ['Qwen/Qwen2.5-7B-Instruct']
Using model: Qwen/Qwen2.5-7B-Instruct


In [4]:
datasets_str = "bdsaglam/musique-mini,answerable,validation"
noise_rate = 1.0
retriever = "hybrid"
temperature = 0.5
max_new_tokens = 1024

print("🔮 Starting MuSiQue evaluation")
print("=" * 50)
print(f"📝 Model: {model}")
print(f"📊 Dataset: {datasets_str} (noise rate: {noise_rate})")
print(f"🔍 Retriever: {retriever}")
print(f"🌡️ Temperature: {temperature}")
print(f"🎯 Max tokens: {max_new_tokens}")
print("=" * 50)

# Load MuSiQue environment
print("🌍 Loading MuSiQue environment...")

vf_env = vf.load_environment(
    env_id="vf-musique",
    datasets_str=datasets_str,
    noise_rate=noise_rate,
    retriever=retriever,
)
print(f"✅ Environment loaded with {len(vf_env.dataset)} examples")

# Use OpenAI-compatible API client (e.g., for vLLM)
print("🤖 Using OpenAI-compatible API client...")

# Run evaluation using the environment
print("🔄 Running evaluation...")
results = vf_env.evaluate(
    openai_client,
    model,
    rollouts_per_example=1,
    sampling_args={"temperature": temperature, "max_tokens": max_new_tokens},
)


🔮 Starting MuSiQue evaluation
📝 Model: Qwen/Qwen2.5-7B-Instruct
📊 Dataset: bdsaglam/musique-mini,answerable,validation (noise rate: 1.0)
🔍 Retriever: hybrid
🌡️ Temperature: 0.5
🎯 Max tokens: 1024
🌍 Loading MuSiQue environment...


Map: 100%|##########| 300/300 [00:00<?, ? examples/s]

Filter:   0%|          | 0/300 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

2025-09-11 11:12:29 - verifiers.envs.MuSiQueEnv - INFO - eval_dataset is not set, falling back to train dataset


✅ Environment loaded with 100 examples
🤖 Using OpenAI-compatible API client...
🔄 Running evaluation...


Evaluating 100 rollouts: 100%|██████████| 100/100 [00:00<00:00, 831.31it/s]


[Trace(trace_id=tr-28e841f76561246c4230c45c1376c095), Trace(trace_id=tr-9a77b9214ea59edc49e226b04e5fa814), Trace(trace_id=tr-d8304aec4d4bf83afdbfc9ed7ebe1fbd), Trace(trace_id=tr-2944937c3639052c87bef4a230e0531a), Trace(trace_id=tr-7b9aef3336a5140cb7f94debad7c5768), Trace(trace_id=tr-a620a85c4f251a92dbaab4385df87745), Trace(trace_id=tr-00cd3a2749b8133c902aab640227667d), Trace(trace_id=tr-280b6a86371e75724c9f4248e4f1b4c7), Trace(trace_id=tr-46ea0400771e476efb469396c5122d77), Trace(trace_id=tr-f5804dc2c2cc80a10beebaac54238ae8)]

In [5]:
results.metrics.keys()

dict_keys(['exact_match_reward', 'f1_reward', 'retrieval_recall_reward', 'retrieval_precision_reward', 'citation_reward', 'format_reward', 'combined_reward'])

In [6]:
ds = vf_env.make_dataset(results)
ds

Dataset({
    features: ['prompt', 'completion', 'answer', 'task', 'reward', 'info', 'exact_match_reward', 'f1_reward', 'retrieval_recall_reward', 'retrieval_precision_reward', 'citation_reward', 'format_reward', 'combined_reward'],
    num_rows: 100
})

In [7]:
ds.to_json("../outputs/musique-eval-results.jsonl", orient="records", lines=True)

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

2797666

In [8]:
df = ds.to_pandas()
df

,prompt,completion,answer,task,reward,info,exact_match_reward,f1_reward,retrieval_recall_reward,retrieval_precision_reward,citation_reward,format_reward,combined_reward
0,[{'content': 'Answer the question based on the...,"[{'content': '<think> To answer this question,...",LeMat revolver,default,0.008333,"{'answers': ['lemat revolver', 'LeMat revolver...",0.0,0.0,0.0,0.000000,0.0,0.333333,0.008333
1,[{'content': 'Answer the question based on the...,"[{'content': '<think> To answer this question,...",Vitor Constâncio,default,0.991667,"{'answers': ['vitor constâncio', 'Vitor Constâ...",1.0,1.0,1.0,1.000000,1.0,0.666667,0.991667
2,[{'content': 'Answer the question based on the...,"[{'content': '', 'role': 'assistant', 'tool_ca...",45,default,0.316667,"{'answers': ['45'], 'docs': [{'body': 'Weber C...",0.0,0.0,1.0,0.666667,0.0,0.000000,0.316667
3,[{'content': 'Answer the question based on the...,"[{'content': '<think> To answer this question,...",three different relationships he had in the past,default,0.200000,{'answers': ['three different relationships he...,0.0,0.0,0.5,0.500000,0.0,1.000000,0.200000
4,[{'content': 'Answer the question based on the...,"[{'content': '<think> To answer this question,...",Michael Bublé,default,0.175000,"{'answers': ['michael bublé', 'Michael Bublé']...",0.0,0.0,0.5,0.500000,0.0,0.000000,0.175000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,[{'content': 'Answer the question based on the...,"[{'content': '<think> To answer this question,...",two alliances,default,0.000000,"{'answers': ['two alliances'], 'docs': [{'body...",0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000
96,[{'content': 'Answer the question based on the...,"[{'content': '<think> To answer this question,...",John Kukuzelis,default,0.175000,"{'answers': ['john kukuzelis', 'John Kukuzelis...",0.0,0.0,0.5,0.500000,0.0,0.000000,0.175000
97,[{'content': 'Answer the question based on the...,"[{'content': '', 'role': 'assistant', 'tool_ca...",Vito Corleone,default,0.275000,"{'answers': ['Vito Andolini', 'vito andolini c...",0.0,0.0,0.5,0.500000,0.5,1.000000,0.275000
98,[{'content': 'Answer the question based on the...,"[{'content': '<think> To answer this question,...",The Île de la Cité,default,0.166667,"{'answers': ['The Île de la Cité', 'the île de...",0.0,0.0,0.5,0.250000,0.0,0.666667,0.166667


In [9]:
df.describe()

,reward,exact_match_reward,f1_reward,retrieval_recall_reward,retrieval_precision_reward,citation_reward,format_reward,combined_reward
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
mean,0.343842,0.120000,0.163818,0.690000,0.560667,0.245000,0.466667,0.343842
std,0.238936,0.326599,0.344011,0.307975,0.255748,0.344473,0.390688,0.238936
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.175000,0.000000,0.000000,0.500000,0.500000,0.000000,0.000000,0.175000
50%,0.275000,0.000000,0.000000,0.500000,0.500000,0.000000,0.333333,0.275000
75%,0.400000,0.000000,0.000000,1.000000,0.666667,0.500000,1.000000,0.400000
max,0.991667,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.991667


In [10]:
row = df.iloc[0]